# Focusing ultrasound behind bone

 - The 3rd Chilean Symposium on Boundary Element Methods – A programming workshop on ultrasound simulations
 - Friday, November 8, 2024
 - Elwin van 't Wout
 - Pontificia Universidad Católica de Chile

The OptimUS library provides functionality to simulate acoustic wave propagation in unbounded domains with homogeneous scatterers. One of the major challenges in focused ultrasound therapy is to target lesions located behind bone. To maximize the acoustic pressure near the focus but keep pressure levels elsewhere low, the transducer instruments need to be optimized accordingly. In the case of array transducers, each piston element can emit a pressure field with a different velocity and phase.

## Import the OptimUS library

Load the OptimUS library.

In [ ]:
import optimus
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

## Specify a spherical section array transducer as acoustic source

The OptimUS library provides different predefined wave fields, among which is a spherical section array transducer. It has 256 elements in a semi-optimal configuration.

The centroids of an example transducer array are stored in a file available in the data folder (https://github.com/optimuslib/optimus/blob/main/notebooks/Data/default_random_array_centroid_locations.dat).

Let us use a frequency of 400 kHz, which is lower than typical operational frequencies but sufficiently high to analyze focussing capabilities with reasonable computational resources.

In [ ]:
frequency = 400e3
a = 0.003
D = 0.18
centroid_file='Data/default_random_array_centroid_locations.dat'
source = optimus.source.create_array(
    frequency, centroid_locations_filename=centroid_file,
    element_radius=a, location=[-D, 0, 0], number_of_point_sources_per_wavelength=2
)

The location of the centroids of the transducers in the array can be retrieved from the source object and plotted in Matplotlib. The focal point at the global origin is visualised by the red marker.

In [ ]:
x, y, z = source.centroid_locations

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(8, 8))
ax = fig.gca(projection='3d')
ax.scatter(x, y, z)
ax.scatter(0, 0, 0, 'r')
ax.set_aspect('equal')
ax.view_init(90, 0)
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")
plt.show()

## Specify the velocities of the transducer elements

Each of the 256 piston elements in the transducer has a complex-valued velocity. The absolute value refers to the speed of the element while the complex part provides its relative phase.

By default, we can use a constant speed across all elements, but each value can be specified individually if needed.

In [ ]:
my_velocities = np.ones(256, dtype='complex')

**TASK** Provide your own array of 256 complex values for the velocities of the transducer elements, such that the focussing capacity behind the rib of the transducer improves.

Let us normalise the velocities such that the sum of the magnitudes is always equal to one hundred.

In [ ]:
my_velocities = my_velocities / np.sum(np.abs(my_velocities))

source.velocity = my_velocities * 100

In [ ]:
print(source.velocity)

## Specify the physical settings for the simulation

In many ultrasound applications there is bone present in the beam path. For example, the liver is located behind the rib cage. Let us simulate this by placing a sphere with a diameter of 15 mm in front of the focus at the global origin. The element size of 1 mm is sufficiently fine to represent the wave phenomena.

In [ ]:
sphere_radius = 7.5e-3
geometry = optimus.geometry.shapes.Sphere(origin=(-0.02,0.005,0), radius=7.5e-3, element_size=1e-3)
material_ext = optimus.material.load_material('water')
material_int = optimus.material.load_material('bone (cortical)')

## Simulate the wave model

The OptimUS library uses the Boundary Element Method to simulate acoustic wave propagation. Let us use a model with default settings.

In [ ]:
model = optimus.model.create_default_model(source, geometry, material_ext, material_int)

After creating the model, it needs to be solved to obtain the surface potentials at the material interface. Depending on the size of the model, this can be computationally expensive. The solution will be stored inside the model object.

In [ ]:
%%time
model.solve()

## Visualisation of the acoustic field

The acoustic field can be calculated in arbitrary points. Let us visualize the field on a grid in the plane $z=0$.

In [ ]:
%%time
postprocess_plane = optimus.postprocess.VisualisePlane(model)
postprocess_plane.create_computational_grid(resolution=(101, 61), bounding_box=(-0.07, 0.07, -0.04, 0.04))
postprocess_plane.compute_fields()

In [ ]:
%matplotlib inline
print("Incident wavefield from transducer:")
figs = optimus.postprocess.plot_pressure_field(postprocess_plane, clim=(-8,8), field="incident", unit="MPa")
print("Total pressure field with scattering at bone:")
figs = optimus.postprocess.plot_pressure_field(postprocess_plane, clim=(-8,8), field="total", unit="MPa")

## Calculate the pressure field in the focus

The geometric focus of the spherical section is at the global origin. Let us calculate the average pressure level in a box around the focus.

In [ ]:
plot_grid = np.mgrid[-0.01:0.01:20j, -0.005:0.005:10j, -0.005:0.005:10j]
points_focus = np.vstack((plot_grid[0].ravel(), plot_grid[1].ravel(), np.zeros(plot_grid[0].size)))
postprocess_focus = optimus.postprocess.VisualiseCloudPoints(model)
postprocess_focus.create_computational_grid(points_focus)
postprocess_focus.compute_fields()
pressure_focus = np.mean(np.abs(postprocess_focus.field.total_field))

**TASK** Maximize the mean pressure level at the focus.

In [ ]:
print("Mean pressure amplitude at focus:", pressure_focus * 1e-6, "MPa.")

Let us calculate the pressure level at the surface of the sphere. These values can be retrieved directly from the BEM solution, which has the Dirichlet and Neumann surface potentials. This pressure should be as small as possible to avoid skin burns.

**TASK** The mean pressure level at the surface has to be lower than 1.5 MPa.

In [ ]:
pressure_sphere = np.mean(np.abs(postprocess_focus.model.solution[0].coefficients))
print("Mean pressure level at the surface of the sphere:", pressure_sphere * 1e-6, "MPa.")

Let us double-check if the sum of the velocities of the transducer array equals one hundred, as should be the case after normalisation.

**TASK** The sum of the velocities has to be equal to one hundred.

In [ ]:
print("Sum of velocities of the transducer array:", np.sum(np.abs(postprocess_focus.model.source.velocity)))